In [10]:
import numpy as np
import polars as pl

df = pl.read_csv('datasets/training_set_VU_DM.csv')

# n = 500000
# new_df = df.head(n)

In [7]:
#delete column if type is string
def delete_string_columns(df):
    for col in df.columns:
        if df[col].dtype==pl.Utf8:
            df = df.drop(col)

    return df

#delete all columns which name ends with _id

def delete_id_columns(df):
    for col in df.columns:
        if col.endswith('_id'):
            df = df.drop(col)
    return df

def delete_defined_columns(df, columns):
    for col in columns:
        df = df.drop(col)
    return df

#rescale all columns to [0,1]
def rescaler(df):
    for col in df.columns:
        # except for the id columns
        if col.endswith('_id') is False:
            df = df.with_columns(
                (pl.col(col) - pl.col(col).min()) / (pl.col(col).max() - pl.col(col).min())
            )
    return df

def save_df(df:pl.DataFrame, path):
    df.write_csv(path)


def main(df:pl.DataFrame, path='')->pl.DataFrame:
    df = delete_string_columns(df)
    #df = delete_id_columns(df)
    df = delete_defined_columns(df, ['site_id', 'visitor_location_country_id', 'prop_country_id', 'srch_destination_id'])
    df = rescaler(df)


    if path:
        save_df(df, path)

    return df


if __name__ == "__main__":
    #read df
    #df = pl.read_csv('datasets/sample_0.1_training_set.csv')
    #df = new_df
    df = pl.read_csv('datasets/training_set_VU_DM.csv')

    #clean df
    df = main(df, path='datasets/clean_0.1_sample.csv')

    print(df)

shape: (4_958_347, 19)
┌─────────┬─────────┬────────────┬────────────┬───┬────────────┬───────────┬──────────┬────────────┐
│ srch_id ┆ prop_id ┆ prop_starr ┆ prop_revie ┆ … ┆ srch_satur ┆ random_bo ┆ click_bo ┆ booking_bo │
│ ---     ┆ ---     ┆ ating      ┆ w_score    ┆   ┆ day_night_ ┆ ol        ┆ ol       ┆ ol         │
│ i64     ┆ i64     ┆ ---        ┆ ---        ┆   ┆ bool       ┆ ---       ┆ ---      ┆ ---        │
│         ┆         ┆ f64        ┆ f64        ┆   ┆ ---        ┆ f64       ┆ f64      ┆ f64        │
│         ┆         ┆            ┆            ┆   ┆ f64        ┆           ┆          ┆            │
╞═════════╪═════════╪════════════╪════════════╪═══╪════════════╪═══════════╪══════════╪════════════╡
│ 1       ┆ 893     ┆ 0.6        ┆ 0.7        ┆ … ┆ 1.0        ┆ 1.0       ┆ 0.0      ┆ 0.0        │
│ 1       ┆ 10404   ┆ 0.8        ┆ 0.8        ┆ … ┆ 1.0        ┆ 1.0       ┆ 0.0      ┆ 0.0        │
│ 1       ┆ 21315   ┆ 0.6        ┆ 0.9        ┆ … ┆ 1.0        ┆ 1.0